In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('LogReg_Toy').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
df_sample = spark.read.format('libsvm').load('data/sample_libsvm_data.txt')
df_sample.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
log_regressor = LogisticRegression()

In [7]:
mod_lr_sample = log_regressor.fit(df_sample)

In [9]:
mod_lr_summary = mod_lr_sample.summary

In [10]:
mod_lr_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



C:\ProgramFiles\pySpark\spark-3.2.1-bin-hadoop3.2\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [11]:
mod_lr_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

C:\ProgramFiles\pySpark\spark-3.2.1-bin-hadoop3.2\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


### Evaluator

In [12]:
df_sample_train, df_sample_test = df_sample.randomSplit([0.7, 0.3])

In [13]:
mod_lr_split = log_regressor.fit(df_sample_train)

In [14]:
mod_lr_split_eval = mod_lr_split.evaluate(df_sample_test)

In [16]:
mod_lr_split_eval.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[18.6582871708331...|[0.99999999211487...|       0.0|
|  0.0|(692,[100,101,102...|[5.04936709972546...|[0.99362747827564...|       0.0|
|  0.0|(692,[123,124,125...|[43.9678574678224...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[106.507773116375...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[30.4212277475677...|[0.99999999999993...|       0.0|
|  0.0|(692,[125,126,127...|[36.6443872118120...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[14.9780623517081...|[0.99999968731284...|       0.0|
|  0.0|(692,[126,127,128...|[17.9821761205243...|[0.99999998449612...|       0.0|
|  0.0|(692,[126,127,128...|[38.4817298425822...|           [1.0,0.0]|       0.0|
|  0.0|(692,[127

C:\ProgramFiles\pySpark\spark-3.2.1-bin-hadoop3.2\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [18]:
bi_eval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label')

In [19]:
bi_metric = bi_eval.evaluate(mod_lr_split_eval.predictions)

C:\ProgramFiles\pySpark\spark-3.2.1-bin-hadoop3.2\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [20]:
bi_metric

1.0